In [2]:
import pandas as pd
from datetime import datetime as dt
import datetime as dat
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
jobs = pd.read_csv("../data/fullsample.csv", nrows=100000)

In [4]:
ce5 = pd.read_csv('../data/slurm_wrapper_ce5.log',
                  header=None,
                  delimiter=' - ',
                  engine='python',
                  nrows=1000)

In [5]:
ce6 = pd.read_csv('../data/slurm_wrapper_ce6.log',
                  header=None,
                  delimiter=' - ',
                  engine='python',
                  nrows=1000
                  )

In [6]:
colnames = ['END', 'USER', 'RETRY', 'TIME', 'RETURNCODE', 'COMMAND']
ce5.columns=colnames
ce6.columns=colnames

In [7]:
ce5['END'] = [dt.strptime(row + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in ce5['END']]
ce5['TIME'] = ce5['TIME'].str.replace('time', '', regex=False).astype(float)
ce5['USER'] = ce5['USER'].str.replace('user', '', regex=False).astype(int)
ce5['RETRY'] = ce5['RETRY'].str.replace('retry', '', regex=False).astype(int)
ce5['RETURNCODE'] = ce5['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
ce5['COMMAND'] = ce5['COMMAND'].str.replace('command', '', regex=False)

In [8]:
ce6['END'] = [dt.strptime(row + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in ce6['END']]
ce6['TIME'] = ce6['TIME'].str.replace('time', '', regex=False).astype(float)
ce6['USER'] = ce6['USER'].str.replace('user', '', regex=False).astype(int)
ce6['RETRY'] = ce6['RETRY'].str.replace('retry', '', regex=False).astype(int)
ce6['RETURNCODE'] = ce6['RETURNCODE'].str.replace('returncode', '', regex=False).astype(float)
ce6['COMMAND'] = ce6['COMMAND'].str.replace('command', '', regex=False)

In [9]:
sbatch = ce6.loc[ce6['COMMAND'].str.contains('/usr/bin/sbatch')]

In [10]:
jobs = jobs[jobs['END'] != 'Unknown']

In [11]:
# Fixes jobs BEGIN and END columns to datetime
jobs['END'] = [dt.strptime(row.replace('T', ' ') + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in jobs['END']]
jobs['BEGIN'] = [dt.strptime(row.replace('T', ' ') + ('.000000' if '.' not in row else ''), '%Y-%m-%d %H:%M:%S.%f') for row in jobs['BEGIN']]

In [12]:
# Fixes USEDMEM column to remove the M and convert to a numeric
jobs['USEDMEM'] = jobs['USEDMEM'].str[:-1]
jobs['USEDMEM'] = pd.to_numeric(jobs['USEDMEM'])

In [13]:
# Cleans memory of nodes/CPUs to allow for numeric manipulation
jobs.insert(5, 'Mc_Mn', jobs['REQMEM'].str[-2:])
jobs['REQMEM']  = jobs['REQMEM'].str[:-2]
jobs['REQMEM'] = pd.to_numeric(jobs['REQMEM'])

In [14]:
# Creates new column that is the difference between the memory used and memory requested
diffmem = jobs['REQMEM'] - jobs['USEDMEM']
jobs.insert(7, 'DIFFMEM', diffmem)

In [16]:
# New column that turns the completion status into a boolean for logistic regression
jobs.insert(2, 'COMPLETE', jobs['STATE'] == 'COMPLETED')
jobs['COMPLETE'] = jobs['COMPLETE'].astype(int)

In [56]:
incomplete = jobs.loc[jobs['COMPLETE'] != 1].head(5)

In [19]:
USEDMEM_pivot = jobs.pivot_table(values='USEDMEM', index=['NODES','EXITCODE', 'STATE'], aggfunc='mean')

In [20]:
REQMEM_pivot = jobs.pivot_table(values='REQMEM', index=['Mc_Mn','EXITCODE', 'STATE'], aggfunc='mean')

The **EXITCODE** gives the [exit code](https://www.agileconnection.com/article/overview-linux-exit-codes) for the job, with "0:0" indicating a successful job. Exit codes have two numbers, where if the first number is non-zero, it indicates a problem on the server side and if the second is nonzero, it indicates a problem on the user side.

In [54]:
diff_mem_group = jobs.groupby(['STATE', 'PARTITION', 'EXITCODE'])['DIFFMEM'].describe()

In [23]:
# jobs['MONTH'] = jobs['END'].dt.month

In [24]:
jobs.groupby('JOBID')['USEDMEM'].mean()

JOBID
30796171       243216.12
30853133        20604.62
30858137        57553.77
30935078        20577.96
31364111_10      9697.47
                 ...    
32002391_95      1638.59
32002391_96      1682.93
32002391_97      1685.29
32002391_98      1689.44
32002391_99      1695.11
Name: USEDMEM, Length: 99993, dtype: float64

In [25]:
jobs[jobs['JOBID'].str.contains('30853133')]

,JOBID,STATE,COMPLETE,BEGIN,END,REQMEM,Mc_Mn,USEDMEM,DIFFMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,1,2021-08-06 11:36:09,2021-09-05 11:36:32,262144,Mn,20604.62,241539.38,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
